<a href="https://colab.research.google.com/github/rafaelhamer/DSWP/blob/master/Labdata_Churn_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import os
for dirname, _, filenames in os.walk('/gdrive/My Drive/Python'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
def mostra_missing_value(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = 100*round((df.isnull().sum()/df.isnull().count()).sort_values(ascending = False), 2)
    missing_data = pd.concat([total, percent], axis = 1, keys=['Total', 'Percentual'])
    print(missing_data.head(10))

## Lendo os dados de Treino e Teste

In [ ]:
df_train = pd.read_csv('/gdrive/My Drive/Python/treino_limpo.csv', index_col='id')
df_test  = pd.read_csv('/gdrive/My Drive/Python/teste_limpo.csv', index_col='id')

In [ ]:
df_train.head(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
id,,,,,,,,,,,,,,,,,,,,
4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0
6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.7,0
6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.8,1
6368,Male,0,No,No,70.0,No,No phone service,DSL,Yes,No,Yes,Yes,No,Yes,Two year,No,Bank transfer (automatic),48.40,3442.8,0
5830,Male,0,Yes,Yes,31.0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,No,Electronic check,59.95,1848.8,0
4476,Female,0,No,No,3.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Electronic check,19.55,61.05,0
1508,Male,0,No,No,54.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),19.65,1008.7,0


In [ ]:
df_train.dtypes

In [ ]:
mostra_missing_value(df_train)

In [ ]:
mostra_missing_value(df_test)

## Transformando variáveis categóricas com `LabelEncoder`

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
for cat_var in df_train.select_dtypes(include='O').columns:
    le = LabelEncoder()
    le.fit(df_train[cat_var])
    df_train[cat_var + '_num'] = le.transform(df_train[cat_var])
    df_train.drop(cat_var, axis=1, inplace=True)
    df_test[cat_var + '_num'] = le.transform(df_test[cat_var])
    df_test.drop(cat_var, axis=1, inplace=True)

In [ ]:
df_train.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_num,Partner_num,Dependents_num,PhoneService_num,MultipleLines_num,InternetService_num,OnlineSecurity_num,OnlineBackup_num,DeviceProtection_num,TechSupport_num,StreamingTV_num,StreamingMovies_num,Contract_num,PaperlessBilling_num,PaymentMethod_num
id,,,,,,,,,,,,,,,,,,,,
4030,0,56.000000,45.05,2560.10,0,0,0,0,0,1,0,2,2,2,2,0,0,2,0,4
6731,0,16.928753,19.65,332.65,0,1,1,1,1,0,2,1,1,1,1,1,1,1,0,3
6479,0,60.000000,110.80,6640.70,0,0,1,0,1,2,1,0,2,2,2,2,2,2,1,1
6861,0,37.000000,101.90,3545.35,1,0,0,0,1,2,1,0,2,2,0,2,2,0,1,0
3266,0,29.000000,70.75,1974.80,1,1,1,1,1,0,1,0,0,0,0,0,0,0,1,2


In [ ]:
df_test.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_num,Partner_num,Dependents_num,PhoneService_num,MultipleLines_num,InternetService_num,OnlineSecurity_num,OnlineBackup_num,DeviceProtection_num,TechSupport_num,StreamingTV_num,StreamingMovies_num,Contract_num,PaperlessBilling_num,PaymentMethod_num
id,,,,,,,,,,,,,,,,,,,
5027,0,23.0,20.00,445.30,1,1,1,1,0,2,1,1,1,1,1,1,1,1,1
1733,1,61.0,99.00,5969.30,1,1,1,1,2,1,0,2,0,0,2,2,1,0,4
5384,0,36.0,84.75,3050.15,1,0,0,1,2,1,0,0,0,0,0,2,0,1,2
6554,0,61.0,61.45,3751.15,0,1,1,0,1,0,0,2,2,2,2,2,2,1,0
364,0,47.0,20.55,945.70,0,0,0,1,0,2,1,1,1,1,1,1,2,1,3


## Eliminando variáveis

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
df_train_limpo = df_train.copy()
df_test_limpo = df_test.copy()

In [ ]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(df_train_limpo)

array([[5.60000000e+01, 4.50500000e+01, 2.56010000e+03, ...,
        2.00000000e+00, 0.00000000e+00, 4.00000000e+00],
       [1.69287532e+01, 1.96500000e+01, 3.32650000e+02, ...,
        1.00000000e+00, 0.00000000e+00, 3.00000000e+00],
       [6.00000000e+01, 1.10800000e+02, 6.64070000e+03, ...,
        2.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       ...,
       [2.80000000e+01, 5.46500000e+01, 1.51750000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.30000000e+01, 2.52500000e+01, 1.57305000e+03, ...,
        2.00000000e+00, 0.00000000e+00, 3.00000000e+00],
       [3.80000000e+01, 9.50000000e+01, 3.60560000e+03, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
df_train_limpo

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_num,Partner_num,Dependents_num,PhoneService_num,MultipleLines_num,InternetService_num,OnlineSecurity_num,OnlineBackup_num,DeviceProtection_num,TechSupport_num,StreamingTV_num,StreamingMovies_num,Contract_num,PaperlessBilling_num,PaymentMethod_num
id,,,,,,,,,,,,,,,,,,,,
4030,0,56.000000,45.05,2560.10,0,0,0,0,0,1,0,2,2,2,2,0,0,2,0,4
6731,0,16.928753,19.65,332.65,0,1,1,1,1,0,2,1,1,1,1,1,1,1,0,3
6479,0,60.000000,110.80,6640.70,0,0,1,0,1,2,1,0,2,2,2,2,2,2,1,1
6861,0,37.000000,101.90,3545.35,1,0,0,0,1,2,1,0,2,2,0,2,2,0,1,0
3266,0,29.000000,70.75,1974.80,1,1,1,1,1,0,1,0,0,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6684,0,43.000000,44.15,1931.30,0,1,0,0,0,1,0,0,0,2,2,0,2,1,0,2
604,0,45.128394,108.65,4903.20,0,1,0,0,1,2,1,0,2,2,2,2,2,2,0,2
4803,0,28.000000,54.65,1517.50,0,0,0,0,1,0,0,0,2,0,2,0,0,0,0,0


In [ ]:
sel

VarianceThreshold(threshold=0.15999999999999998)

## Salvando o dataframe tratado

In [ ]:
df_train.to_csv('/gdrive/My Drive/Python/treino_limpo.csv')
df_test.to_csv('/gdrive/My Drive/Python/teste_limpo.csv')

## Treinando o Modelo

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(max_depth=5)

X_train = df_train.drop('Churn', axis=1)
y_train = df_train['Churn']

dt.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
my_predictions = dt.predict(df_test)

## Submetendo a Solução

In [ ]:
submission = pd.read_csv('/gdrive/My Drive/Python/sample_submission.csv')
submission['Churn'] = my_predictions
submission.to_csv('/gdrive/My Drive/Python/ChurnDecisionTree2.csv', index=False)
submission.head(10)

,id,Churn
0,5027,0
1,1733,0
2,5384,0
3,6554,0
4,364,0
5,2093,0
6,6966,0
7,5076,0
8,5746,0
9,5461,0
